<a href="https://colab.research.google.com/github/GuilhermeOchoa/Data-Science/blob/master/Jogo_da_Velha_entre_IA's_20241109.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
class Debugger:
    def __init__(self, enabled=False):
        self.enabled = enabled

    def set_debug(self, enabled):
        """Habilita ou desabilita a impressão de variáveis de depuração."""
        self.enabled = enabled

    def debug_print(self, *args, **kwargs):
        """Imprime apenas se o modo de depuração estiver habilitado."""
        if self.enabled:
            print(*args, **kwargs)


In [48]:
import random

class Sucessor:
    def __init__(self, estado, valor, coluna=None):
        self.estado = estado
        self.valor = valor
        self.coluna = coluna

    def get_estado(self):
        return self.estado

    def get_valor(self):
        return self.valor

    def get_coluna(self):
        return self.coluna

    def __str__(self):
        return f"Estado: {self.estado}, Valor: {self.valor}, Coluna: {self.coluna}"

class Minimax:
    def __init__(self, estado):
        self.estado = estado

    def get_melhor(self):
        melhor_sucessor = self.algoritmo(self.estado, False, self.livres(self.estado))
        return melhor_sucessor.coluna if melhor_sucessor else None  # Retorna o índice da melhor jogada

    def get_melhor_ab(self):
        melhor_sucessor = self.algoritmo_ab(self.estado, False, self.livres(self.estado), -999, 999)
        return melhor_sucessor.coluna if melhor_sucessor else None  # Retorna o índice da melhor jogada

    def livres(self, estado):
        return estado.count('0')

    def jogada_aleatoria(self):
        posicoes_livres = [i for i, v in enumerate(self.estado) if v == '0']
        return random.choice(posicoes_livres) if posicoes_livres else None

    def gera_vizinhos(self, estado, caracter):
        vizinhos = []
        for i in range(9):
            if estado[i] == '0':
                novo_estado = estado[:]
                novo_estado[i] = caracter
                vizinhos.append((novo_estado, i))  # Retorna o estado e o índice da jogada
        return vizinhos

    def utilidade(self, atual, profundidade):
        if self.vencedor(atual, 'X'):
            return -1
        if self.vencedor(atual, 'O'):
            return 1
        if profundidade == 0:
            return 0
        return 100

    def vencedor(self, atual, caracter):
        # Verifica linhas, colunas e diagonais para vitória
        for i in range(3):
            if all(atual[i*3 + j] == caracter for j in range(3)) or all(atual[i + j*3] == caracter for j in range(3)):
                return True
        return (atual[0] == caracter and atual[4] == caracter and atual[8] == caracter) or \
               (atual[2] == caracter and atual[4] == caracter and atual[6] == caracter)

    def algoritmo(self, estado, jogador, profundidade):
        valor = self.utilidade(estado, profundidade)
        if valor != 100:
            return Sucessor(estado, valor)

        vizinhos = self.gera_vizinhos(estado, 'X' if jogador else 'O')
        melhor_sucessor = None
        if jogador:
            menor = 999
            for vizinho, indice in vizinhos:
                atual = self.algoritmo(vizinho, False, profundidade - 1)
                if atual.get_valor() < menor:
                    menor = atual.get_valor()
                    melhor_sucessor = Sucessor(vizinho, menor, coluna=indice)  # Armazena o índice
            return melhor_sucessor
        else:
            maior = -999
            for vizinho, indice in vizinhos:
                atual = self.algoritmo(vizinho, True, profundidade - 1)
                if atual.get_valor() > maior:
                    maior = atual.get_valor()
                    melhor_sucessor = Sucessor(vizinho, maior, coluna=indice)  # Armazena o índice
            return melhor_sucessor

    def algoritmo_ab(self, estado, jogador, profundidade, alfa, beta):
        valor = self.utilidade(estado, profundidade)
        if valor != 100:
            return Sucessor(estado, valor)

        vizinhos = self.gera_vizinhos(estado, 'X' if jogador else 'O')
        melhor_sucessor = None
        if jogador:
            menor = 999
            for vizinho, indice in vizinhos:
                atual = self.algoritmo_ab(vizinho, False, profundidade - 1, alfa, beta)
                if atual.get_valor() < menor:
                    menor = atual.get_valor()
                    melhor_sucessor = Sucessor(vizinho, menor, coluna=indice)  # Armazena o índice
                if menor < alfa:
                    return melhor_sucessor
                beta = min(beta, menor)
            return melhor_sucessor
        else:
            maior = -999
            for vizinho, indice in vizinhos:
                atual = self.algoritmo_ab(vizinho, True, profundidade - 1, alfa, beta)
                if atual.get_valor() > maior:
                    maior = atual.get_valor()
                    melhor_sucessor = Sucessor(vizinho, maior, coluna=indice)  # Armazena o índice
                if maior > beta:
                    return melhor_sucessor
                alfa = max(alfa, maior)
            return melhor_sucessor


In [87]:
# Definição da Rede Neural
import numpy as np
import random

# Função de ativação
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Definição da Rede Neural
class MLP:
    def __init__(self, input_size, hidden_size, output_size, population_size, generations, mutation_rate, debugger: Debugger):
        # Instancia o algoritmo genético para gerar os pesos iniciais
        self.genetic_algorithm = GeneticAlgorithm(input_size, hidden_size, output_size, population_size, generations, mutation_rate,debugger)

        # Obtém os pesos iniciais da população do algoritmo genético
        self.hidden_weights, self.output_weights = self.genetic_algorithm.get_initial_weights()

        self.debugger = debugger

    # Passo de inferência para escolher a jogada
    def forward(self, inputs):
        inputs = np.append(inputs, 1)  # Adiciona bias
        self.debugger.debug_print("Entradas da rede neural: ",inputs)
        hidden_inputs = np.dot(self.hidden_weights, inputs)
        #self.debugger.debug_print("Pesos nas camadas hidden: ", self.hidden_weights)
        hidden_outputs = sigmoid(hidden_inputs)
        self.debugger.debug_print("Pesos * sigmoid  nas Saídas da camada oculta: ",hidden_outputs)
        hidden_outputs = np.append(hidden_outputs, 1)  # Adiciona bias
        #self.debugger.debug_print(" Saídas da camada oculta: ",hidden_outputs)
        final_inputs = np.dot(self.output_weights, hidden_outputs)
       # self.debugger.debug_print("Pesos da camada de saída: ",self.output_weights)
        final_outputs = sigmoid(final_inputs)
        self.debugger.debug_print("Saídas da rede neural: ",final_outputs)
        return final_outputs

    # Escolhe a jogada baseada na saída da rede neural
    def escolher_jogada(self, tabuleiro):
        inputs = np.array([1 if pos == 'O' else 0 for pos in tabuleiro])
        outputs = self.forward(inputs)
        self.debugger.debug_print("Saídas da rede neural: ",outputs)
        melhor_jogada = np.argmax(outputs)
        self.debugger.debug_print("Melhor jogada:", melhor_jogada)
        return melhor_jogada

    # Atualiza os pesos com base no resultado do jogo
    def atualizar_pesos(self, game_result, invalid_moves):
        # Chama o método de refinamento da classe GeneticAlgorithm com o resultado do jogo
        self.hidden_weights, self.output_weights = self.genetic_algorithm.refinar_pesos(
            self.hidden_weights, self.output_weights, game_result, invalid_moves
        )

In [94]:
# Algoritmo Genético
import numpy as np
import random

class GeneticAlgorithm:
    def __init__(self, input_size, hidden_size, output_size, population_size, generations, mutation_rate,debugger: Debugger):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.population = self.initialize_population()
        self.debugger = debugger
    def initialize_population(self):
        population = []
        for _ in range(self.population_size):
            hidden_weights = np.random.uniform(-1, 1, (self.hidden_size, self.input_size + 1))
            output_weights = np.random.uniform(-1, 1, (self.output_size, self.hidden_size + 1))
            population.append((hidden_weights, output_weights))
        return population

    def get_initial_weights(self):
        initial_hidden, initial_output = self.population[0]
        self.debugger.debug_print("Pesos iniciais da camada oculta:", initial_hidden)
        self.debugger.debug_print("Pesos iniciais da camada de saída:", initial_output)
        return initial_hidden, initial_output

    def fitness_function(self, hidden_weights, output_weights, game_result, invalid_moves):
        score = 0
        if game_result == 1: # Ganhou
            score += 10
        elif game_result == 0: # Empatou
            score += 5
        elif game_result == -1: # Perdeu
            score -= 5
        score -= invalid_moves * 2  # Penaliza movimentos inválidos
        return score

    # Seleção, cruzamento e mutação da população
    def refinar_pesos(self, hidden_weights, output_weights, game_result, invalid_moves):
        fitness_scores = []

        # Avalia a população existente
        for chromo_hidden, chromo_output in self.population:
            score = self.fitness_function(chromo_hidden, chromo_output, game_result, invalid_moves)
            fitness_scores.append(score)

        # Ordena a população com base na aptidão
        sorted_population = [self.population[i] for i in np.argsort(fitness_scores)[::-1]]

        # Seleciona os melhores para a próxima geração usando elitismo
        new_population = sorted_population[:self.population_size // 10]

        # Realiza torneios para preencher o restante da população
        while len(new_population) < self.population_size:
            parent1, parent2 = random.choice(sorted_population), random.choice(sorted_population)
            child_hidden, child_output = self.crossover(parent1, parent2)
            self.mutate(child_hidden)
            self.mutate(child_output)
            new_population.append((child_hidden, child_output))

        # Atualiza a população e escolhe o melhor conjunto de pesos
        self.population = new_population
        best_hidden, best_output = sorted_population[0]
        return best_hidden, best_output

    # Cruzamento entre dois cromossomos
    def crossover(self, parent1, parent2):
        child_hidden = (parent1[0] + parent2[0]) / 2
        child_output = (parent1[1] + parent2[1]) / 2
        return child_hidden, child_output

    # Mutação com pequena alteração em alguns pesos
    def mutate(self, weight_matrix):
        mutation_mask = np.random.rand(*weight_matrix.shape) < self.mutation_rate
        weight_matrix += mutation_mask * np.random.uniform(-0.5, 0.5, weight_matrix.shape)


In [ ]:


import random
def inicializar_tabuleiro():
    return ['0' for _ in range(9)]

def imprimir_tabuleiro(tabuleiro):
    print(f"{tabuleiro[0]} | {tabuleiro[1]} | {tabuleiro[2]}")
    print("-" * 10)
    print(f"{tabuleiro[3]} | {tabuleiro[4]} | {tabuleiro[5]}")
    print("-" * 10)
    print(f"{tabuleiro[6]} | {tabuleiro[7]} | {tabuleiro[8]}")

def verificar_estado(tabuleiro):
    # Verifica as linhas
    for i in range(0, 9, 3):
        if tabuleiro[i] == tabuleiro[i+1] == tabuleiro[i+2] and tabuleiro[i] != '0':
            return 2 if tabuleiro[i] == 'X' else 1  # 2 para MLP (X), 1 para Minimax (O)

    # Verifica as colunas
    for i in range(3):
        if tabuleiro[i] == tabuleiro[i+3] == tabuleiro[i+6] and tabuleiro[i] != '0':
            return 2 if tabuleiro[i] == 'X' else 1  # 1 para MLP (X), 2 para Minimax (O)

    # Verifica as diagonais
    if tabuleiro[0] == tabuleiro[4] == tabuleiro[8] and tabuleiro[0] != '0':
        return 2 if tabuleiro[0] == 'X' else 1
    if tabuleiro[2] == tabuleiro[4] == tabuleiro[6] and tabuleiro[2] != '0':
        return 2 if tabuleiro[2] == 'X' else 1

    # Verifica se houve empate
    if all(x != '0' for x in tabuleiro):  # Corrige '0' para string
        return 3  # Empate

    return 0  # Jogo continua

def jogada_ia_MiniMax(tabuleiro, dificuldade):
    jogo_ia = Minimax(tabuleiro)

    if dificuldade == 1:  # Nível 1: 25% Minimax, 75% Aleatório
        probabilidade_minimax = 0.25
    elif dificuldade == 2:  # Nível 2: 50% Minimax, 50% Aleatório
        probabilidade_minimax = 0.5
    else:  # Nível 3: 100% Minimax
        probabilidade_minimax = 1.0

    if random.random() < probabilidade_minimax:
        movimento_ia = jogo_ia.get_melhor()  # Pega o índice da jogada
    else:
        movimento_ia = jogo_ia.jogada_aleatoria()

    tabuleiro[movimento_ia] = 'O'
    print(f"IA MiniMax escolheu a posição {movimento_ia}")

def jogada_ia_MLP(tabuleiro, mlp):
    movimento_iaMLP = mlp.escolher_jogada(tabuleiro)
    if tabuleiro[movimento_iaMLP] != '0':  # Verifica se a jogada é inválida
        print(f"Jogada inválida pela IA MLP na posição {movimento_iaMLP}. Penalizando...")
        invalid_move = True
    else:
        tabuleiro[movimento_iaMLP] = 'X'
        print(f"IA MLP escolheu a posição {movimento_iaMLP}")
        invalid_move = False
    return invalid_move  # Retorna se a jogada foi inválida

def jogada_humana(tabuleiro):
    while True:
        try:
            posicao = int(input("Escolha uma posição de 0 a 8: "))
            if tabuleiro[posicao] == '0' and 0 <= posicao <= 8:
                tabuleiro[posicao] = 'X'
                break
            else:
                print("Posição inválida. Tente novamente.")
        except ValueError:
            print("Entrada inválida. Por favor, insira um número de 0 a 8.")



def jogar():
  continuar_jogando = True

  while continuar_jogando:
    print("Escolha uma opção de jogo:")
    print("1 - Jogar contra Minimax")
    print("2 - Treinar a rede neural jogando contra Minimax")
    print("3 - Jogar contra a rede neural treinada")

    try:
        escolha = int(input("Digite o número da opção escolhida: "))
        if escolha not in [1, 2, 3]:
          print("Escolha inválida. Tente novamente.")
          continue
    except ValueError:
          print("Escolha inválida. Tente novamente.")
          continue

    num_partidas = int(input("Quantas partidas você deseja jogar? "))
    dificuldade = int(input("Escolha a dificuldade (1: Fácil, 2: Médio, 3: Difícil): "))
    debugger = Debugger(enabled=True)
    mlp = MLP(input_size=9, hidden_size=9, output_size=9, population_size=50, generations=100, mutation_rate=0.1, debugger=debugger)


    # Contadores para vitórias e empates
    vitorias_jogador = 0
    vitorias_minimax = 0
    vitorias_mlp = 0
    empates = 0

    for partida in range(num_partidas):
        print(f"\n--- Partida {partida + 1} ---")
        tabuleiro = inicializar_tabuleiro()
        jogadas = 0
        invalid_moves = 0

        while True:
            imprimir_tabuleiro(tabuleiro)

            if escolha == 1:
                if jogadas % 2 == 0:
                    jogada_humana(tabuleiro)
                else:
                    jogada_ia_MiniMax(tabuleiro, dificuldade)
            elif escolha == 2:
                if jogadas % 2 == 0:
                    invalid_move = jogada_ia_MLP(tabuleiro, mlp)
                    if invalid_move:
                        invalid_moves += 1  # Incrementa penalidade para MLP
                else:
                    jogada_ia_MiniMax(tabuleiro, dificuldade)
            elif escolha == 3:
                if jogadas % 2 == 0:
                    jogada_humana(tabuleiro)
                else:
                    jogada_ia_MLP(tabuleiro,mlp)

            estado = verificar_estado(tabuleiro)
            debugger.debug_print("Estado do jogo:",estado)
            if estado == 1:
                imprimir_tabuleiro(tabuleiro)
                print("IA Minimax venceu!")
                vitorias_minimax += 1
                mlp.atualizar_pesos(-1, invalid_moves)  # Penaliza a MLP por derrota e jogadas inválidas
                break
            elif estado == 2:
                imprimir_tabuleiro(tabuleiro)
                print("Jogador / IA MLP venceu!")
                vitorias_mlp += 1
                mlp.atualizar_pesos(1, invalid_moves)  # Reforça a MLP por vitória, mas penaliza jogadas inválidas
                break
            elif estado == 3:
                imprimir_tabuleiro(tabuleiro)
                print("Empate!")
                empates += 1
                mlp.atualizar_pesos(0, invalid_moves)  # Penaliza jogadas inválidas no caso de empate
                break

            jogadas += 1

    print("\n--- Resultado Final ---")
    print(f"Vitórias Jogador / IA MLP: {vitorias_mlp}")
    print(f"Vitórias da IA Minimax: {vitorias_minimax}")
    print(f"Empates: {empates}")

# Pergunta se deseja jogar novamente
    while True:
            resposta = input("Deseja jogar novamente? (s/n): ").lower()
            if resposta in ['s', 'n']:
                break
            else:
                print("Por favor, digite 's' para sim ou 'n' para não.")

  continuar = resposta == 's'
# Executa o jogo
jogar()


A saída de streaming foi truncada nas últimas 5000 linhas.
Saídas da rede neural:  [0.58859865 0.27952915 0.70115998 0.52647752 0.66695686 0.38872951
 0.61941969 0.71505513 0.44717562]
Saídas da rede neural:  [0.58859865 0.27952915 0.70115998 0.52647752 0.66695686 0.38872951
 0.61941969 0.71505513 0.44717562]
Melhor jogada: 7
Jogada inválida pela IA MLP na posição 7. Penalizando...
Estado do jogo: 0
O | O | 0
----------
0 | 0 | O
----------
O | X | 0
IA MiniMax escolheu a posição 2
Estado do jogo: 1
O | O | O
----------
0 | 0 | O
----------
O | X | 0
IA Minimax venceu!

--- Partida 370 ---
0 | 0 | 0
----------
0 | 0 | 0
----------
0 | 0 | 0
Entradas da rede neural:  [0 0 0 0 0 0 0 0 0 1]
Pesos * sigmoid  nas Saídas da camada oculta:  [0.49458406 0.51418912 0.55302805 0.43923028 0.57025028 0.49262474
 0.66031061 0.41390059 0.59322392]
Saídas da rede neural:  [0.55485656 0.43165758 0.54359523 0.59234896 0.58883214 0.31884673
 0.59984948 0.80781628 0.54912091]
Saídas da rede neural:  [0.5